# Index Repr Demo

This notebook demonstrates the `__repr__` and `_repr_html_` methods for the custom indexes in this package.

Both `DimensionInterval` and `NDIndex` provide informative representations that:
- Show the structure of the index
- Display dimensions and coordinates being managed
- Include value ranges and other metadata
- Support both light and dark modes
- Provide expandable sections with selection examples

The HTML representations are automatically used in Jupyter notebooks for rich display.

In [1]:
import numpy as np
import xarray as xr
from IPython.display import display, HTML

from linked_indices import DimensionInterval, NDIndex
from linked_indices.example_data import multi_interval_dataset, trial_based_dataset

---

## DimensionInterval Repr

The `DimensionInterval` index manages multiple interval dimensions over a single continuous dimension.

### Standard repr (text)

In [2]:
# Create a dataset with DimensionInterval
ds = multi_interval_dataset()
ds = ds.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_intervals",
        "phoneme_intervals",
        "word",
        "part_of_speech",
        "phoneme",
    ],
    DimensionInterval,
)

# Get the index
di_index = ds.xindexes["time"]

In [3]:
# Standard text repr
print(repr(di_index))

<DimensionInterval>
  Continuous: time
    size: 1000, range: [0, 120]
  Interval dimensions:
    word:
      coord: word_intervals
      size: 3, range: [0, 120), closed: 'left'
      labels: ['word', 'part_of_speech']
    phoneme:
      coord: phoneme_intervals
      size: 6, range: [0, 120), closed: 'left'
      labels: ['phoneme']


### HTML repr (rich display)

The HTML representation shows the same information in a styled table format with:
- Color-coded elements for easy reading
- Expandable sections for interval details and selection examples
- Dark/light mode support

In [4]:
# HTML repr - automatically used in Jupyter
# Click on "Show interval details" and "Selection examples" to expand!
display(HTML(di_index._repr_html_()))

### Interactive Selection Demo

Let's try the selection examples shown in the repr:

In [5]:
# Select a time range (from the selection examples)
result = ds.sel(time=slice(0, 60))
print("Original time range: 0-120")
print(
    f"Selected time range: {float(result.time.min()):.1f}-{float(result.time.max()):.1f}"
)
print(f"Words in range: {list(result.word.values)}")
print(f"Phonemes in range: {list(result.phoneme.values)}")

Original time range: 0-120
Selected time range: 0.0-59.9
Words in range: [np.str_('red'), np.str_('green')]
Phonemes in range: [np.str_('ah'), np.str_('ee'), np.str_('oh')]


In [6]:
# Select by word label
result = ds.sel(word="green")
print(f"Selected word: {result.word.values}")
print(
    f"Time range for 'green': {float(result.time.min()):.1f}-{float(result.time.max()):.1f}"
)
print(f"Phonemes in 'green': {list(result.phoneme.values)}")

Selected word: ['green']
Time range for 'green': 40.0-80.0
Phonemes in 'green': [np.str_('oh'), np.str_('oo')]


In [7]:
# Select by part of speech
result = ds.sel(part_of_speech="noun")
print(f"Words with part_of_speech='noun': {list(result.word.values)}")
print(f"Time range: {float(result.time.min()):.1f}-{float(result.time.max()):.1f}")

Words with part_of_speech='noun': [np.str_('blue')]
Time range: 80.0-120.0


In [8]:
# Select by interval coordinate value
result = ds.sel(word_intervals=50)  # Time point 50 falls in which word interval?
print(f"Time 50 is in word: {result.word.values}")
print(f"That word's interval: {result.word_intervals.values}")

Time 50 is in word: ['green']
That word's interval: [Interval(40.0, 80.0, closed='left')]


---

### DimensionInterval with onset/duration format

When intervals are constructed from onset/duration coordinates, the repr indicates this:

In [9]:
# Create dataset with onset/duration format
N = 1000
times = np.linspace(0, 120, N)

ds_onset = xr.Dataset(
    {"data": (("C", "time"), np.random.rand(2, N))},
    coords={
        "time": times,
        "word_onset": ("word", [0.0, 40.0, 80.0]),
        "word_duration": ("word", [35.5, 35.5, 35.5]),
        "word": ("word", ["hello", "world", "test"]),
    },
)

ds_onset = ds_onset.drop_indexes(["time", "word"]).set_xindex(
    ["time", "word_onset", "word_duration", "word"],
    DimensionInterval,
    onset_duration_coords={"word": ("word_onset", "word_duration")},
)

onset_index = ds_onset.xindexes["time"]

In [10]:
# Text repr shows "(from onset/duration)"
print(repr(onset_index))

<DimensionInterval>
  Continuous: time
    size: 1000, range: [0, 120]
  Interval dimensions:
    word: (from onset/duration)
      size: 3, range: [0, 115.5), closed: 'left'
      labels: ['word_onset', 'word_duration', 'word']


In [11]:
# HTML repr
display(HTML(onset_index._repr_html_()))

In [12]:
# Try selecting by word label
result = ds_onset.sel(word="world")
print(f"Selected word: {result.word.values}")
print(f"Onset: {float(result.word_onset)}, Duration: {float(result.word_duration)}")
print(f"Time range: {float(result.time.min()):.1f}-{float(result.time.max()):.1f}")

Selected word: ['world']
Onset: 40.0, Duration: 35.5
Time range: 40.0-75.4


---

## NDIndex Repr

The `NDIndex` manages N-dimensional derived coordinates (like `abs_time` with shape `(trial, rel_time)`).

### Standard repr (text)

In [13]:
# Create a dataset with NDIndex
ds_nd = trial_based_dataset(n_trials=3, trial_length=5.0, sample_rate=10)
ds_nd = ds_nd.set_xindex(["abs_time"], NDIndex)

# Get the index
nd_index = ds_nd.xindexes["abs_time"]

In [14]:
# Standard text repr
print(repr(nd_index))

<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 14.9]


### HTML repr (rich display)

The HTML repr includes expandable sections for:
- **Selection examples**: Copy-paste code for common selection patterns
- **Value preview**: See the actual coordinate values

In [15]:
# HTML repr with expandable sections
display(HTML(nd_index._repr_html_()))

### Interactive Selection Demo

Let's try the selection examples from the repr:

In [16]:
# Scalar selection (nearest) - finds the cell where abs_time ≈ 7.45
result = ds_nd.sel(abs_time=7.45, method="nearest")
print("Query: abs_time=7.45")
print(f"Found trial: {result.trial.values}")
print(f"Found rel_time: {float(result.rel_time):.2f}")
print(f"Actual abs_time: {float(result.abs_time):.2f}")

Query: abs_time=7.45
Found trial: ['square']
Found rel_time: 2.40
Actual abs_time: 7.40


In [17]:
# Slice selection - finds bounding box containing all cells in range
result = ds_nd.sel(abs_time=slice(3.725, 11.18))
print("Query: abs_time=slice(3.725, 11.18)")
print(f"Trials in result: {list(result.trial.values)}")
print(f"Shape: {dict(result.sizes)}")
print(
    f"Actual abs_time range: [{float(result.abs_time.min()):.2f}, {float(result.abs_time.max()):.2f}]"
)

Query: abs_time=slice(3.725, 11.18)
Trials in result: [np.str_('cosine'), np.str_('square'), np.str_('sawtooth')]
Shape: {'trial': 3, 'rel_time': 50}
Actual abs_time range: [0.00, 14.90]


In [18]:
# Slice with step - subsamples the inner dimension
result = ds_nd.sel(abs_time=slice(0, 14.9, 2))
print("Query: abs_time=slice(0, 14.9, 2)")
print(f"Original rel_time size: {ds_nd.sizes['rel_time']}")
print(f"Result rel_time size: {result.sizes['rel_time']} (every 2nd point)")

Query: abs_time=slice(0, 14.9, 2)
Original rel_time size: 50
Result rel_time size: 25 (every 2nd point)


---

### NDIndex with trim_outer slice method

The slice_method configuration is shown in the repr:

In [19]:
# Create with trim_outer slice method
ds_trim = trial_based_dataset(n_trials=3, trial_length=5.0, sample_rate=10)
ds_trim = ds_trim.set_xindex(["abs_time"], NDIndex, slice_method="trim_outer")

trim_index = ds_trim.xindexes["abs_time"]
print(repr(trim_index))

<NDIndex>
  slice_method: 'trim_outer'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 14.9]


In [20]:
display(HTML(trim_index._repr_html_()))

---

### NDIndex with multiple coordinates

When multiple N-D coordinates are managed by the same index, all are shown:

In [21]:
# Create with multiple 2D coordinates
ds_multi = trial_based_dataset(n_trials=3, trial_length=5.0, sample_rate=10)
ds_multi = ds_multi.assign_coords(
    {"normalized_time": ds_multi.abs_time / float(ds_multi.abs_time.max())}
)
ds_multi = ds_multi.set_xindex(["abs_time", "normalized_time"], NDIndex)

multi_index = ds_multi.xindexes["abs_time"]
print(repr(multi_index))

<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 14.9]
    normalized_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 1]


In [22]:
display(HTML(multi_index._repr_html_()))

In [23]:
# Can select by either coordinate
print("Select by abs_time=7.5:")
r1 = ds_multi.sel(abs_time=7.5, method="nearest")
print(f"  trial={r1.trial.values}, rel_time={float(r1.rel_time):.2f}")

print("\nSelect by normalized_time=0.5:")
r2 = ds_multi.sel(normalized_time=0.5, method="nearest")
print(f"  trial={r2.trial.values}, rel_time={float(r2.rel_time):.2f}")

Select by abs_time=7.5:
  trial=['square'], rel_time=2.50

Select by normalized_time=0.5:
  trial=['square'], rel_time=2.50


---

### NDIndex with 3D coordinates

NDIndex works with coordinates of any dimensionality >= 2:

In [24]:
# Create 3D dataset: subject × trial × rel_time
n_subjects, n_trials, n_times = 2, 3, 50

subjects = ["alice", "bob"]
trials = ["trial_0", "trial_1", "trial_2"]
rel_time = np.linspace(0, 5, n_times)

# Subject session offsets
subject_offset = xr.DataArray(
    [0.0, 100.0], dims=["subject"], coords={"subject": subjects}
)

# Trial onsets
trial_onset = xr.DataArray([0.0, 10.0, 20.0], dims=["trial"], coords={"trial": trials})

# 3D absolute time
abs_time_3d = subject_offset + trial_onset + xr.DataArray(rel_time, dims=["rel_time"])

ds_3d = xr.Dataset(
    {
        "signal": (
            ("subject", "trial", "rel_time"),
            np.random.randn(n_subjects, n_trials, n_times),
        )
    },
    coords={
        "subject": subjects,
        "trial": trials,
        "rel_time": rel_time,
        "abs_time": abs_time_3d,
    },
)

ds_3d = ds_3d.set_xindex(["abs_time"], NDIndex)
index_3d = ds_3d.xindexes["abs_time"]
print(repr(index_3d))

<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (subject, trial, rel_time)
      shape: (2 × 3 × 50)
      range: [0, 125]


In [25]:
display(HTML(index_3d._repr_html_()))

In [26]:
# 3D selection example
# Find the cell where abs_time ≈ 112.5 (Bob's trial_1, middle of time)
result = ds_3d.sel(abs_time=112.5, method="nearest")
print("Query: abs_time=112.5")
print(f"Found subject: {result.subject.values}")
print(f"Found trial: {result.trial.values}")
print(f"Found rel_time: {float(result.rel_time):.2f}")
print(f"Actual abs_time: {float(result.abs_time):.2f}")

Query: abs_time=112.5
Found subject: ['bob']
Found trial: ['trial_1']
Found rel_time: 2.45
Actual abs_time: 112.45


---

## Repr after slicing

The repr updates to reflect the current state of the index after slicing operations:

In [27]:
# Slice DimensionInterval dataset
ds_sliced = ds.sel(time=slice(20, 60))

print("DimensionInterval after slicing time to [20, 60]:")
print(repr(ds_sliced.xindexes["time"]))

DimensionInterval after slicing time to [20, 60]:
<DimensionInterval>
  Continuous: time
    size: 333, range: [20.06, 59.94]
  Interval dimensions:
    word:
      coord: word_intervals
      size: 2, range: [0, 80), closed: 'left'
      labels: ['word', 'part_of_speech']
    phoneme:
      coord: phoneme_intervals
      size: 2, range: [20, 60), closed: 'left'
      labels: ['phoneme']


In [28]:
display(HTML(ds_sliced.xindexes["time"]._repr_html_()))

In [29]:
# Slice NDIndex dataset
ds_nd_sliced = ds_nd.isel(trial=slice(0, 2))

print("NDIndex after slicing to first 2 trials:")
print(repr(ds_nd_sliced.xindexes["abs_time"]))

NDIndex after slicing to first 2 trials:
<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (2 × 50)
      range: [0, 9.9]


In [30]:
display(HTML(ds_nd_sliced.xindexes["abs_time"]._repr_html_()))

---

## Dark Mode Support

The HTML repr automatically adapts to your Jupyter theme:

- Uses `prefers-color-scheme` CSS media query for system-wide dark mode
- Detects JupyterLab's theme via `[data-jp-theme-light="false"]`

Try switching your JupyterLab theme to see the repr adapt!

---

## Summary

Both index classes provide informative representations:

### DimensionInterval repr shows:
- **Continuous dimension**: name, size, and value range
- **Interval dimensions**: 
  - Dimension name
  - Coordinate name (or "from onset/duration" indicator)
  - Size, range, and closed property
  - Associated label coordinates
- **Expandable details**: Individual intervals with labels
- **Selection examples**: Copy-paste code snippets

### NDIndex repr shows:
- **Slice method**: 'bounding_box' or 'trim_outer'
- **N-D coordinates**:
  - Coordinate name
  - Dimensions tuple
  - Shape
  - Value range
- **Selection examples**: Scalar, slice, and step selections
- **Value preview**: Actual coordinate values

### Features:
- **Dark/light mode**: Automatic theme adaptation
- **Interactive**: Expandable sections with `<details>` elements
- **Copy-paste code**: Selection examples you can run directly